In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


In [2]:
fraud=pd.read_csv("/content/sample_data/Fraud_check.csv")
fraud.head()

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [3]:
fraud = fraud.replace(r'^\s*$', np.nan, regex=True)

EDA


In [4]:
fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [5]:
fraud=fraud.rename({'Marital.Status':'Marital_status',
                   'Taxable.Income':'Taxable_income',
                   'City.Population':'City_Population',
                   'Work.Experience':'Work_experience'},axis=1)

In [6]:
fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital_status   600 non-null    object
 2   Taxable_income   600 non-null    int64 
 3   City_Population  600 non-null    int64 
 4   Work_experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [7]:
fraud.describe().T

,count,mean,std,min,25%,50%,75%,max
Taxable_income,600.0,55208.375000,26204.827597,10003.0,32871.50,55074.5,78611.75,99619.0
City_Population,600.0,108747.368333,49850.075134,25779.0,66966.75,106493.5,150114.25,199778.0
Work_experience,600.0,15.558333,8.842147,0.0,8.00,15.0,24.00,30.0


In [8]:
#making object datatype as categorifor feature in fraud.columns:cal
for feature in fraud.columns:
  if fraud[feature].dtype=='object':
   fraud[feature]=pd.Categorical(fraud[feature])

fraud.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Undergrad        600 non-null    category
 1   Marital_status   600 non-null    category
 2   Taxable_income   600 non-null    int64   
 3   City_Population  600 non-null    int64   
 4   Work_experience  600 non-null    int64   
 5   Urban            600 non-null    category
dtypes: category(3), int64(3)
memory usage: 16.3 KB


In [9]:
fraud.head()

,Undergrad,Marital_status,Taxable_income,City_Population,Work_experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [10]:
print(fraud['Undergrad'].value_counts())
print("**************************************")
print(fraud['Marital_status'].value_counts())
print("**************************************")


YES    312
NO     288
Name: Undergrad, dtype: int64
**************************************
Single      217
Married     194
Divorced    189
Name: Marital_status, dtype: int64
**************************************


In [11]:
y=fraud[['Taxable_income']]
y.head()

,Taxable_income
0,68833
1,33700
2,36925
3,50190
4,81002


In [12]:
#risky=1 good =0
y=np.where(y.Taxable_income>=30000,0,1)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

In [13]:
x=fraud.drop('Taxable_income',axis=1)
x.head()

,Undergrad,Marital_status,City_Population,Work_experience,Urban
0,NO,Single,50047,10,YES
1,YES,Divorced,134075,18,YES
2,NO,Married,160205,30,YES
3,YES,Single,193264,15,YES
4,NO,Married,27533,28,NO


In [14]:
oneHotCols=["Undergrad","Marital_status","Urban"]

In [15]:
x=pd.get_dummies(x, columns=oneHotCols)
x.head(10)

,City_Population,Work_experience,Undergrad_NO,Undergrad_YES,Marital_status_Divorced,Marital_status_Married,Marital_status_Single,Urban_NO,Urban_YES
0,50047,10,1,0,0,0,1,0,1
1,134075,18,0,1,1,0,0,0,1
2,160205,30,1,0,0,1,0,0,1
3,193264,15,0,1,0,0,1,0,1
4,27533,28,1,0,0,1,0,1,0
5,116382,0,1,0,1,0,0,1,0
6,80890,8,1,0,1,0,0,0,1
7,131253,3,0,1,0,0,1,0,1
8,102481,12,1,0,0,0,1,0,1
9,155482,4,0,1,1,0,0,0,1


In [16]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [17]:
#lets go with gini criterial : higher the gini index better the split
DT_model1=DecisionTreeClassifier(criterion='gini',random_state=1)
DT_model1.fit(x_train,y_train)

DecisionTreeClassifier(random_state=1)

In [18]:
print(DT_model1.score(x_train,y_train))
print(DT_model1.score(x_test,y_test))

1.0
0.6416666666666667


In [19]:
for max_depth in range (2,5):
  DT_model=DecisionTreeClassifier(criterion='gini',max_depth=max_depth,random_state=42)
  DT_model.fit(x_train,y_train)
  print("max_depth = {}".format(max_depth))
  print(DT_model.score(x_train,y_train))
  print(DT_model.score(x_test,y_test))
  print(pd.DataFrame(DT_model.feature_importances_ , columns=['imp'],index=x.columns))
  print("***********************************")

max_depth = 2
0.7958333333333333
0.7833333333333333
                              imp
City_Population          0.358297
Work_experience          0.342416
Undergrad_NO             0.000000
Undergrad_YES            0.000000
Marital_status_Divorced  0.000000
Marital_status_Married   0.299287
Marital_status_Single    0.000000
Urban_NO                 0.000000
Urban_YES                0.000000
***********************************
max_depth = 3
0.80625
0.775
                              imp
City_Population          0.680865
Work_experience          0.224929
Undergrad_NO             0.000000
Undergrad_YES            0.000000
Marital_status_Divorced  0.000000
Marital_status_Married   0.094206
Marital_status_Single    0.000000
Urban_NO                 0.000000
Urban_YES                0.000000
***********************************
max_depth = 4
0.8083333333333333
0.7833333333333333
                              imp
City_Population          0.683389
Work_experience          0.253585
Undergrad_NO  

In [20]:
print(DT_model.feature_importances_)

[0.6833893  0.25358519 0.         0.         0.         0.06302552
 0.         0.         0.        ]


In [21]:
print(pd.DataFrame(DT_model.feature_importances_ , columns=['imp'],index=x.columns))

                              imp
City_Population          0.683389
Work_experience          0.253585
Undergrad_NO             0.000000
Undergrad_YES            0.000000
Marital_status_Divorced  0.000000
Marital_status_Married   0.063026
Marital_status_Single    0.000000
Urban_NO                 0.000000
Urban_YES                0.000000


remove unwanted column


In [22]:
fraud.head()

,Undergrad,Marital_status,Taxable_income,City_Population,Work_experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [23]:
x.head()

,City_Population,Work_experience,Undergrad_NO,Undergrad_YES,Marital_status_Divorced,Marital_status_Married,Marital_status_Single,Urban_NO,Urban_YES
0,50047,10,1,0,0,0,1,0,1
1,134075,18,0,1,1,0,0,0,1
2,160205,30,1,0,0,1,0,0,1
3,193264,15,0,1,0,0,1,0,1
4,27533,28,1,0,0,1,0,1,0


In [24]:
x1=x.drop(['Undergrad_NO','Undergrad_YES','Marital_status_Divorced','Marital_status_Single','Urban_NO','Urban_YES'],axis=1)
x1.head()

,City_Population,Work_experience,Marital_status_Married
0,50047,10,0
1,134075,18,0
2,160205,30,1
3,193264,15,0
4,27533,28,1


In [25]:
x_train,x_test,y_train,y_test=train_test_split(x1,y,test_size=0.2,random_state=42)

In [26]:
for i in range (2,5):
  DT_model=DecisionTreeClassifier(criterion='entropy',max_depth=i,random_state=42)
  DT_model.fit(x_train,y_train)
  print("max_depth = {}".format(i))
  print(DT_model.score(x_train,y_train))
  print(DT_model.score(x_test,y_test))
  print(pd.DataFrame(DT_model.feature_importances_ , columns=['imp'],index=x1.columns))
  print("***********************************")

max_depth = 2
0.7958333333333333
0.7833333333333333
                             imp
City_Population         0.380301
Work_experience         0.318174
Marital_status_Married  0.301526
***********************************
max_depth = 3
0.80625
0.775
                             imp
City_Population         0.662406
Work_experience         0.245990
Marital_status_Married  0.091603
***********************************
max_depth = 4
0.80625
0.7833333333333333
                             imp
City_Population         0.630114
Work_experience         0.312172
Marital_status_Married  0.057715
***********************************


In [27]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
x_new=x.values
x_new

array([[ 50047,     10,      1, ...,      1,      0,      1],
       [134075,     18,      0, ...,      0,      0,      1],
       [160205,     30,      1, ...,      0,      0,      1],
       ...,
       [154058,      0,      1, ...,      0,      0,      1],
       [180083,     17,      0, ...,      0,      1,      0],
       [158137,     16,      1, ...,      0,      1,      0]])

In [29]:
y_new=y
y_new

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

In [30]:
num_fold=5
kfold=KFold(n_splits=num_fold)
model=DecisionTreeClassifier(criterion='entropy',max_depth=3,random_state=5)
results=cross_val_score(model,x_new,y_new,cv=kfold)
results*100

array([77.5       , 73.33333333, 75.83333333, 77.5       , 85.83333333])

In [32]:
results.mean()*100

78.0